In [9]:
import os
import h5py
import numpy as np

data_path = os.path.join('.','tests','test_output','data.hdf5')

dat = h5py.File(data_path, 'r')

m = dat.get('data').shape[0]

m_c = np.sort(np.random.choice(range(0,m), 3000, replace = False))

X = np.array(dat.get('data')[m_c,:])

dat.close()



[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [ ]:
from sklearn.svm import SVC
import numpy as np
from sklearn import decomposition
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
import os
import h5py
import pandas as pd
import multiprocessing as mp

data_path = os.path.join('.','tests','test_output','data.hdf5')

dat = h5py.File(data_path, 'r')

X = np.array(dat.get('data'))
Y = np.array(dat.get('data_label'))

Cs = np.logspace(-1,2,5)
pca_red = np.logspace(1.38,np.log10(64**3)/np.log10(10),10).astype(int)

mean_lin_df = pd.DataFrame(index = pca_red,columns=Cs)
var_lin_df = pd.DataFrame(index = pca_red,columns=Cs)


#for c in Cs:
#    for comp in pca_red:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

linear_svc = SVC(C = 10, kernel = 'linear',gamma = 'auto')
linear_svc.fit(X_train,y_train)

score_lin = linear_svc.score(X_test,y_test)

print(score_lin)
        
dat.close()



In [17]:
import os
import h5py
import numpy as np

data_path = os.path.join('.','tests','test_output','data.hdf5')

dat = h5py.File(data_path, 'r')

m = dat.get('data').shape[0]

m_c = np.sort(np.random.choice(range(0,m), 3000, replace = False)).tolist()

X = np.array(dat.get('data')[m_c,:])
Y = np.array(dat.get('data_label')[m_c])

dat.close()

In [19]:
from multiprocessing.dummy import Pool
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA
from sklearn.svm import SVC

def opti_func(train_i,test_i,X,Y,c,n):
    X_train = X[train_i]
    X_test = X[test_i]
    y_train = Y[train_i]
    y_test = Y[test_i]
    
    pca = PCA(n_components=n)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
    
    linear_svc = SVC(C = c, kernel = 'linear',gamma = 'auto')
    linear_svc.fit(X_train,y_train)
    
    return linear_svc.score(X_test,y_test)
    
rkf = RepeatedKFold(n_splits=10, n_repeats=1)
split = rkf.split(X)

t_pool = Pool(10)

results = [t_pool.apply(opti_func, args = (train,test,X,Y,100,1000)) for train, test in split]

print(results)


[0.5133333333333333, 0.52, 0.5, 0.4533333333333333, 0.48333333333333334, 0.5433333333333333, 0.5733333333333334, 0.58, 0.45666666666666667, 0.45]
